In [1]:
# Main import block
import asyncio
import datetime
import signal
import time
import warnings
from contextlib import contextmanager
from datetime import date, datetime
from io import StringIO

# Sneaky scrapper
import cfscrape
# Upload to google sheets
import gspread
# import df2gspread as d2g
import gspread_pandas as gsp
import numpy as np
import pandas as pd
import requests
# Upload to yandex
import yadisk
from bs4 import BeautifulSoup
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

warnings.filterwarnings("ignore")

In [2]:
################################################################
####################   CB_currencies ###########################
################################################################


async def cb_currency():
    async def get_currency(currency):
        current_date = datetime.today().strftime("%d.%m.%Y")

        dict_of_currencies = {
            "USD": "R01235",
            "EUR": "R01239",
            "Australian_Dollar": "R01010",
            "China_Yuan": "R01375",
            "British_Pound": "R01035",
            "Kazakhstan_Tenge": "R01335",
            "Japanese_Yen": "R01820",
            "Swiss_Franc": "R01775",
        }

        url = f"https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.\
Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.\
date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={dict_of_currencies[currency]}&UniDbQuery.\
From=01.01.2005&UniDbQuery.To={current_date}"

        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        return scraped_data

    def data_reconstruction(scraped_data, name):

        # Получаем датафрейм
        data = pd.read_html(scraped_data.text.replace(",", "."))[0]
        data.columns = data.iloc[1]

        # Преобразуем
        data = data.iloc[2:]
        data = data.rename(
            columns={
                data.columns[0]: "date",
                data.columns[1]: "unit",
                data.columns[2]: "nominal",
            }
        ).reset_index(drop=True)

        # Cкидываем название индекса
        data = pd.read_csv(StringIO(data.to_string(index=False)), delim_whitespace=True)
        data["date"] = pd.to_datetime(data["date"], format="%d.%m.%Y")
        data["unit"] = pd.to_numeric(data["unit"])
        data["nominal"] = pd.to_numeric(data["nominal"])

        data = data.sort_values("date").reset_index(drop=True)

        print(f"{name} is done!")
        display(data.tail())

        return data

    try:
        raw_usd = await get_currency(currency="USD")
        # raw_eur = await get_currency(currency="EUR")
        # raw_ausd = await get_currency(currency="Australian_Dollar")
        # raw_yuan = await get_currency(currency="China_Yuan")
        # raw_pound = await get_currency(currency="British_Pound")
        # raw_tenge = await get_currency(currency="Kazakhstan_Tenge")
        # raw_yen = await get_currency(currency="Japanese_Yen")
        # raw_swiss = await get_currency(currency="Swiss_Franc")

        usd = data_reconstruction(raw_usd, name="USD")
        # eur = data_reconstruction(raw_eur, name="EUR")
        # ausd = data_reconstruction(raw_ausd, name="Australian_Dollar")
        # yuan = data_reconstruction(raw_yuan, name="China_Yuan")
        # pound = data_reconstruction(raw_pound, name="British_Pound")
        # tenge = data_reconstruction(raw_tenge, name="Kazakhstan_Tenge")
        # yen = data_reconstruction(raw_yen, name="Japanese_Yen")
        # swiss = data_reconstruction(raw_swiss, name="Swiss_Franc")

        with pd.ExcelWriter(
            "./data/cb_currency_avg.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            usd.to_excel(writer, sheet_name="USD")
            # eur.to_excel(writer, sheet_name="EUR")
            # ausd.to_excel(writer, sheet_name="Australian_Dollar")
            # yuan.to_excel(writer, sheet_name="China_Yuan")
            # pound.to_excel(writer, sheet_name="British_Pound")
            # tenge.to_excel(writer, sheet_name="Kazakhstan_Tenge")
            # yen.to_excel(writer, sheet_name="Japanese_Yen")
            # swiss.to_excel(writer, sheet_name="Swiss_Franc")

    except Exception as error:
        print(error)
        

In [3]:
await cb_currency()

USD is done!


,date,unit,nominal
4776,2024-04-24,1,93.2918
4777,2024-04-25,1,92.5058
4778,2024-04-26,1,92.1314
4779,2024-04-27,1,92.0134
4780,2024-04-28,1,91.7791


In [19]:
raw_data = pd.read_excel("./data/cb_currency_avg.xlsx", index_col=0)

In [25]:
raw_data.set_index("date", inplace=True)

In [50]:
yearly_mean_asis = raw_data.resample('Y').mean()

In [52]:
yearly_mean_asis.to_excel("./data/yearly_mean_asis.xlsx")

In [ ]:
raw_data

In [45]:
df = raw_data.resample('D').asfreq()

df['nominal'] = df['nominal'].interpolate(method='pad')

df["unit"] = df["unit"].fillna(1)

In [46]:
df.to_excel("./data/cb_currency_avg_interpolated.xlsx")

In [47]:
yearly_mean = df.resample('Y').mean()

In [48]:
yearly_mean

,unit,nominal
date,,
2005-12-31,1.0,28.286376
2006-12-31,1.0,27.185247
2007-12-31,1.0,25.577000
2008-12-31,1.0,24.855280
2009-12-31,1.0,31.723096
2010-12-31,1.0,30.369221
2011-12-31,1.0,29.387439
2012-12-31,1.0,31.092953
2013-12-31,1.0,31.847986


In [53]:
yearly_mean.to_excel("./data/yearly_mean_interpolate.xlsx")